<a href="https://colab.research.google.com/github/senju-sudharsan/DATA-SCIENCE-PROJECTS/blob/main/Live_Sentiment_Analyzer_Real_Time_Text_Emotion_Detection_with_Streamlit_%26_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tweepy transformers nltk pandas matplotlib seaborn wordcloud streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.2 MB/s eta 0:00:00


Saving training.1600000.processed.noemoticon.csv to training.1600000.processed.noemoticon.csv


In [8]:
!pip install transformers pandas matplotlib wordcloud
import pandas as pd
# Load data
df = pd.read_csv('training.1600000.processed.noemoticon.csv',
                 encoding='latin-1',
                 header=None,
                 names=["target", "id", "date", "flag", "user", "text"])

# Map target to labels
df["label"] = df["target"].map({4: "POSITIVE", 0: "NEGATIVE"})

# Analyze sentiment with Hugging Face
from transformers import pipeline
analyzer = pipeline("sentiment-analysis")

# Sample 100 tweets for speed
results = []
for text in df["text"].sample(100):
    try:
        sentiment = analyzer(text[:512])[0]  # Trim to model's max length
        results.append({"text": text, "predicted": sentiment["label"], "actual": df[df["text"] == text]["label"].values[0]})
    except:
        pass

results_df = pd.DataFrame(results)
results_df.head()

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


,text,predicted,actual
0,In loving memory of my Grandfather Louis Dale ...,POSITIVE,NEGATIVE
1,"nothing satisfied my munchies, and now my tumm...",NEGATIVE,NEGATIVE
2,@arwen04 no news just need to talk to people ...,NEGATIVE,NEGATIVE
3,loving the new phone.....,POSITIVE,POSITIVE
4,working party tonite rhough!,NEGATIVE,NEGATIVE


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import pipeline
import streamlit as st
from wordcloud import WordCloud
from google.colab import files
import io


In [10]:
# Initialize the sentiment analysis pipeline from Hugging Face
analyzer = pipeline("sentiment-analysis")

# Function to analyze sentiment of text
def analyze_sentiment(text):
    sentiment = analyzer(text[:512])[0]  # Ensure the text is not longer than 512 tokens
    return sentiment['label'], sentiment['score']


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [12]:
# Write the Streamlit app to a Python file (dashboard.py)
%%writefile dashboard.py
import streamlit as st
from transformers import pipeline
from wordcloud import WordCloud

# Initialize sentiment analysis pipeline
analyzer = pipeline("sentiment-analysis")

# Streamlit app content
def display_dashboard():
    st.title('Real-Time Sentiment Analysis Dashboard')

    st.header("Enter Text for Sentiment Analysis:")
    user_input = st.text_area("Input Text Here:")

    if user_input:
        sentiment, score = analyzer(user_input[:512])[0]  # Ensure the text is not longer than 512 tokens
        st.write(f"Sentiment: {sentiment}")
        st.write(f"Confidence: {score:.2f}")

        # Display WordCloud for the sentiment
        wordcloud = WordCloud(width=400, height=400, background_color='white').generate(user_input)
        st.image(wordcloud.to_array())

# Run the dashboard when called
if __name__ == "__main__":
    display_dashboard()



Writing dashboard.py


In [13]:
!pip install pyngrok
from pyngrok import ngrok

# Run the Streamlit app in the background
!streamlit run dashboard.py &

# Create a public URL using ngrok
public_url = ngrok.connect(port='8501')
print(public_url)





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.185.53.177:8501

  Stopping...


ERROR:pyngrok.process.ngrok:t=2025-04-21T06:44:32+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-04-21T06:44:32+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-04-21T06:44:32+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [ ]:
# Streamlit interface for real-time sentiment analysis
def display_dashboard():
    st.title('Real-Time Sentiment Analysis Dashboard')

    st.header("Enter Text for Sentiment Analysis:")
    user_input = st.text_area("Input Text Here:")

    if user_input:
        sentiment, score = analyze_sentiment(user_input)
        st.write(f"Sentiment: {sentiment}")
        st.write(f"Confidence: {score:.2f}")

        # Visualize sentiment over time
        sentiment_label = 'Positive' if sentiment == 'POSITIVE' else 'Negative'
        st.markdown(f"### Sentiment Trend: {sentiment_label}")

        # Display WordCloud for the sentiment
        wordcloud = WordCloud(width=400, height=400, background_color='white').generate(user_input)
        st.image(wordcloud.to_array())

# Run Streamlit application within Colab
!streamlit run dashboard.py


Dashboard is live at: http://xxxxxx.ngrok.io